In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


True

## Without any output parser

In [6]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D53673F460>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D536760070>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

c:\Users\Sahil\Desktop\GenAI_Cousera\langchain\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sahil\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
template1=PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variables=['topic']
)

template2=PromptTemplate(
    template="Write a 5 line summary on the following text. /n {text}",
    input_variables=['text']
)
prompt1=template1.invoke({'topic':'black hole'})  ## we can also do template1.foramte()

result=model.invoke(prompt1)

prompt2=template2.invoke({'text':result.content})

result1=model.invoke(prompt2)
print(result1)


content='Here is a 5-line summary of the text:\n\nBlack holes are regions of spacetime with incredibly high density and gravitational pull, where nothing, not even light, can escape once it falls within a certain distance. They are formed when a massive star collapses in on itself, and have several key characteristics, including an event horizon, singularity, and strong gravitational pull. There are four types of black holes, including stellar, supermassive, intermediate-mass, and primordial black holes. Black holes can be detected through their effects on the surrounding environment, such as X-rays, gamma rays, and gravitational waves. Ongoing research aims to shed light on the formation, properties, and consequences of black holes, and to test our understanding of the universe.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 1074, 'total_tokens': 1227, 'completion_time': 0.1275, 'prompt_time': 0.298200642, 'queue_time': 0.252805362,

## String ouput parser

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

chain=template1|model|parser|template2|model|parser

result=chain.invoke({'topic':'black hole'})
result

'Here is a 5-line summary of the text:\n\nBlack holes are regions of spacetime with incredibly strong gravitational pull, where nothing can escape once it crosses the event horizon. They come in various sizes, from stellar black holes formed from star collapse to supermassive black holes at galaxy centers. Black holes have properties such as mass, charge, and spin, and their behavior is influenced by their accretion disks, jets, and winds. The study of black holes has led to important advances in our understanding of gravity, spacetime, and the behavior of matter and energy. Black holes play a crucial role in the evolution of the universe, influencing the formation of galaxies and the distribution of matter.'

## JSON Output parser

In [11]:
from langchain_core.output_parsers import JsonOutputParser
parser=JsonOutputParser()
template1=PromptTemplate(
    template='Give me the name, age city of a fictional person\n{format_instruction}',
    input_variables=[],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

promt=template1.format()
result=model.invoke(promt)
final_result=parser.parse(result.content)
final_result

{'name': 'Evelyn Starling', 'age': 32, 'city': 'Portland'}

In [12]:
chain=template1|model|parser
result=chain.invoke({})
result

{'name': 'Evelyn Stone', 'age': 32, 'city': 'Portland'}

draw back of jsonout parser-> you doesnot force a schema


## Structured output parser

output->json but here we can provide a schema

In [16]:
from langchain.output_parsers import StructuredOutputParser,ResponseSchema

schema=[
    ResponseSchema(name='fact_1',description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2',description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3',description='Fact 3 about the topic'),
]


template=PromptTemplate(
    template='Give 3 facts about {topic} \n {formate_instruction}',
    input_variables=['topic'],
    partial_variables={'formate_instruction':parser.get_format_instructions()}

)

parser=StructuredOutputParser.from_response_schemas(schema)
prompt=template.invoke({'topic':'black hole'})
result=model.invoke(prompt)
final_result=parser.parse(result.content)
final_result

{'fact_1': 'Black holes are regions in space where the gravitational pull is so strong that nothing, including light, can escape. They are formed when a massive star collapses in on itself and its gravity becomes so strong that it warps the fabric of spacetime.',
 'fact_2': "The event horizon of a black hole is the point of no return. Once something crosses the event horizon, it is trapped by the black hole's gravity and cannot escape, even if it's moving at the speed of light.",
 'fact_3': 'Black holes come in various sizes, ranging from small, stellar-mass black holes formed from the collapse of individual stars, to supermassive black holes found at the centers of galaxies, with masses millions or even billions of times that of the sun.'}

In [17]:
chain=template|model|parser
result=chain.invoke({'topic':'black hole'})
result

{'fact_1': 'Black holes are regions in space where the gravitational pull is so strong that nothing, including light, can escape once it falls inside.',
 'fact_2': 'The event horizon of a black hole is the point of no return, where the gravitational pull is so strong that anything that crosses it will be consumed by the black hole.',
 'fact_3': 'The size of a black hole is typically measured by its mass, spin, and charge, and can range from small, stellar-mass black holes formed from the collapse of individual stars, to supermassive black holes found at the centers of galaxies.'}

we can not do Data validation in StructuredOutputParser

## Pydantic Output parser

In [18]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

In [19]:
class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

final_result = chain.invoke({'place':'sri lankan'})

print(final_result)

name='Tharindu Silva' age=25 city='Colombo'
